In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv

In [4]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
print('x_train:', x_train.shape)

K = len(np.unique(y_train)) # Classes
Ntr = x_train.shape[0]
Nte = x_test.shape[0]
Din = 3072 # CIFAR10
# Din = 784 # MINIST

# Normalize pixel values
x_train, x_test = x_train / 255.0, x_test / 255.0
mean_image = np.mean(x_train, axis=0)
x_train = x_train - mean_image
x_test = x_test - mean_image
y_train = tf.keras.utils.to_categorical(y_train, num_classes=K)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=K)
x_train = np.reshape(x_train,(Ntr,Din))
x_test = np.reshape(x_test,(Nte,Din))
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

std=1e-6
w1 = std*np.random.randn(Din, K)
b1 = np.zeros(K)
print("w1:", w1.shape)
print("b1:", b1.shape)

x_train: (50000, 32, 32, 3)
w1: (3072, 10)
b1: (10,)


In [5]:
#define variables 
batch_size = Ntr
iterations = 300
lr = 0.01
lr_decay= 0.99
reg = 5e-6
loss_history = []
train_acc_history = []
val_acc_history = []

In [6]:
def calc_loss(y, y_pred, w1, w2 = 0):
    batch_size=y_pred.shape[0]
    loss = 1./batch_size*np.square(y_pred - y).sum() + reg*(np.sum(w2*w2) + np.sum(w1*w1))
    return loss

In [7]:
def accuracy(y_pred,y):
    batch_size=y_pred.shape[0]
    K=y_pred.shape[1]
    acc=1-(1/(batch_size*K))*(np.abs(np.argmax(y,axis=1)-np.argmax(y_pred,axis=1))).sum()
    return acc

In [8]:
#linear classifier
def classifier(x_train,y_train,x_test,y_test,K,Din,lr,lr_decay,reg):
    Ntr = x_train.shape[0]
    Nte = x_test.shape[0]

    loss_history = []
    train_acc_history = []
    val_acc_history = []
    seed = 0
    rng = np.random.default_rng(seed=seed)

    std=1e-5
    w1 = std*np.random.randn(Din, K)
    b1 = np.zeros(K)

    for t in range(iterations):
        indices = np.arange(Ntr)
        rng.shuffle(indices)
        x=x_train[indices]
        y=y_train[indices]

        #forward pass
        y_pred=x.dot(w1)+b1
        y_pred_test=x_test.dot(w1)+b1
        train_loss=calc_loss(y_pred,y,w1)
        test_loss=calc_loss(y_pred_test,y_test,w1)
        loss_history.append(train_loss)
        
        train_acc=accuracy(y_pred,y)
        train_acc_history.append(train_acc)
        test_acc=accuracy(y_pred_test,y_test)
        val_acc_history.append(test_acc)

        if t%10 == 0:
            print('epoch %d/%d: train loss= %f ,test loss= %f,train accracy= %f, test accracy= %f' % (t,iterations,train_loss,test_loss,train_acc,test_acc))

        # Backward pass
        dy_pred=(1./batch_size)*2.0*(y_pred-y)
        dw1=x.T.dot(dy_pred)+reg*w1
        db1=dy_pred.sum(axis=0)
        w1-=lr*dw1
        b1-=lr*db1
        lr*=lr_decay
    return w1,b1,loss_history,train_acc_history,val_acc_history

In [9]:
w1,b1,loss_history,train_acc_history,val_acc_history=classifier(x_train, y_train, x_test, y_test, K, Din, lr, lr_decay, reg)

epoch 0/300: train loss= 1.000011 ,test loss= 1.000010,train accracy= 0.663282, test accracy= 0.666790
epoch 10/300: train loss= 0.893986 ,test loss= 0.893440,train accracy= 0.751962, test accracy= 0.754560
epoch 20/300: train loss= 0.863943 ,test loss= 0.863635,train accracy= 0.759154, test accracy= 0.762200
epoch 30/300: train loss= 0.845984 ,test loss= 0.845828,train accracy= 0.761924, test accracy= 0.767420
epoch 40/300: train loss= 0.834138 ,test loss= 0.834102,train accracy= 0.764218, test accracy= 0.769190
epoch 50/300: train loss= 0.825929 ,test loss= 0.825997,train accracy= 0.765990, test accracy= 0.770740
epoch 60/300: train loss= 0.820035 ,test loss= 0.820196,train accracy= 0.767184, test accracy= 0.771230
epoch 70/300: train loss= 0.815678 ,test loss= 0.815924,train accracy= 0.767896, test accracy= 0.771580
epoch 80/300: train loss= 0.812377 ,test loss= 0.812701,train accracy= 0.768620, test accracy= 0.771590
epoch 90/300: train loss= 0.809821 ,test loss= 0.810217,train acc

In [10]:
x_axis=np.arange(len(loss_history))
plt.plot(x_axis,loss_history)

In [11]:
import cv2 as cv
images=[]
for i in range(w1.shape[1]):
    #images.append(np.reshape(w1[:,i]*255,(32,32,3))+np.reshape(mean_image,(32,32,3)))
    temp=np.reshape(w1[:,i]*255,(32,32,3))
    temp=cv.normalize(temp, None, 0, 255, cv.NORM_MINMAX, cv.CV_8U)
    images.append(temp)
fig,ax=plt.subplots(2,5,figsize=(30,10))
for i in range(2):
    for j in range(5):
        ax[i,j].imshow(images[i*5+j],vmin=0,vmax=255)
plt.show()